# Pointnet - CIFAR 10

we will using cifar-10 as our dataset to do the classfication work

Since the image are 1024(32x32) point only, we will choosing 512 points

## Prepare datasets

### 1. Downdload datasets

In [1]:
import os
import sys
import numpy as np
import h5py

BASE_DIR = '/notebooks/'
# Download dataset for point cloud classification
DATA_DIR = os.path.join(BASE_DIR, 'data')
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)
if not os.path.exists(os.path.join(DATA_DIR, 'cifar-10-python-py')):
    www = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

    zipfile = os.path.basename(www)
    os.system('wget --no-check-certificate  %s; tar -xzf %s' % (www, zipfile))
    os.system('mv cifar-10-batches-py %s' % DATA_DIR+'/cifar-10-python-py')
    os.system('rm cifar-10-python.tar.gz')


### 2. prepare h5 files

In [2]:
import h5py
import numpy as np


# Write numpy array data and label to h5_filename
def save_h5_data_label(h5_filename, data, label, data_dtype='float32', label_dtype='uint8'):
    h5_fout = h5py.File(h5_filename)
    h5_fout.create_dataset(
            'data', data=data,
            compression='gzip', compression_opts=4,
            dtype=data_dtype)
    h5_fout.create_dataset(
            'label', data=label,
            compression='gzip', compression_opts=1,
            dtype=label_dtype)
    h5_fout.close()

def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

In [3]:
train_batchs = [
    'data/cifar-10-python-py/data_batch_1',
    'data/cifar-10-python-py/data_batch_2',
    'data/cifar-10-python-py/data_batch_3',
    'data/cifar-10-python-py/data_batch_4',
    'data/cifar-10-python-py/data_batch_5',
]

test_batch = [
    'data/cifar-10-python-py/test_batch',
]

In [4]:
def transform2pointcloud(file_path):
    """
    transfrom iamge to cloud points
    """
    pickle = unpickle(file_path)
    label = pickle['labels']
    data = pickle['data']
    
    data_arr = data.reshape([-1, 3, 32, 32])
    data_arr = np.transpose(data_arr, (0, 2, 3, 1))
    data_arr = data_arr.astype(np.float32)
    # normalize the data with -0.5, 0.5
    data_arr = data_arr/255. - 0.5
    data_arr_size = data_arr.shape
    
    data = np.zeros((data_arr_size[0], data_arr_size[1], data_arr_size[2], 2+3), np.float32)
    data[:,:,:,-3:] = data_arr
    
    index_arr = np.array(list(np.ndindex((32,32)))).reshape((32,32,2)).astype(np.float32)/32 - 0.5
    index_arr = np.expand_dims(index_arr, axis=0)
    index_arr = np.repeat(index_arr, [data_arr_size[0]], axis=0)
    data[:,:,:,0:2] = index_arr  
    data = data.reshape((data_arr_size[0], data_arr_size[1]*data_arr_size[2],  (2+3)))
    
    return data, np.array(label)

In [5]:
%matplotlib inline

data, label = transform2pointcloud(train_batchs[0])
print(data.shape, label.shape)
print(data.dtype, label.dtype)
# from matplotlib import pyplot as plt
# plt.imshow(data[4])
print('x,y,z,r,g,b', data[4,1,1])

((10000, 1024, 5), (10000,))
(dtype('float32'), dtype('int64'))
('x,y,z,r,g,b', -0.46875)


In [6]:

txt_name = 'data/cifar-10-python-py/train_files.txt'
with open(txt_name, 'w') as file:
    for idx, raw_file in enumerate(train_batchs):
        file_path = 'data/cifar-10-python-py/batch_'+str(idx)+'.h5'
        print(file_path)
        if not os.path.exists(file_path):
            data, labels = transform2pointcloud(raw_file)
            save_h5_data_label(file_path, data, label)
        file.write(file_path)
        file.write('\n')
os.chmod(txt_name, 0o777)
    

txt_name = 'data/cifar-10-python-py/test_files.txt'
with open(txt_name, 'w') as file:
    for idx, raw_file in enumerate(test_batch):
        file_path = 'data/cifar-10-python-py/test_batch_'+str(idx)+'.h5'
        print(file_path)
        if not os.path.exists(file_path):
            data, labels = transform2pointcloud(raw_file)
            save_h5_data_label(file_path, data, label)
        file.write(file_path)
        file.write('\n')
os.chmod(txt_name, 0o777)

data/cifar-10-python-py/batch_0.h5
data/cifar-10-python-py/batch_1.h5
data/cifar-10-python-py/batch_2.h5
data/cifar-10-python-py/batch_3.h5
data/cifar-10-python-py/batch_4.h5
data/cifar-10-python-py/test_batch_0.h5


## View datasets

In [7]:

BASE_DIR = '/notebooks/'
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(os.path.join(BASE_DIR, 'utils'))
import cls_provider as provider
import os 

def savePoints(filename, points):
    """
        filename: string asb path
        points: npArray nbPts:n
    """    
    with open(filename, 'w') as file:
        for row in points:
            for idx, col in enumerate(row):
                if idx >2:
                    file.write(str(int(col))+' ')
#                     pass
                else:
                    file.write(str(col-0.5)+' ')
            file.write('\n')
    os.chmod(filename, 0o777)

In [8]:
# ModelNet40 official train/test split
TRAIN_FILES = provider.getDataFiles( \
    os.path.join(BASE_DIR, 'data/cifar-10-python-py/train_files.txt'))
TEST_FILES = provider.getDataFiles(\
    os.path.join(BASE_DIR, 'data/cifar-10-python-py/test_files.txt'))

In [9]:
# Shuffle train files
train_file_idxs = np.arange(0, len(TRAIN_FILES))
np.random.shuffle(train_file_idxs)

NUM_POINT = 1024

fn = 1

current_data, current_label = provider.loadDataFile(TRAIN_FILES[train_file_idxs[fn]])
current_data = current_data[:,0:NUM_POINT,:]
current_data, current_label, _ = provider.shuffle_data(current_data, np.squeeze(current_label))            
current_label = np.squeeze(current_label)

color_image = current_data[0,:,:]
print(color_image[100])
color_image[:,3:] = (color_image[:,3:] + 0.5) *255
print(color_image[100])

savePoints('image.txt',color_image)


[-0.40625    -0.375       0.08431375  0.14313728  0.19411767]
[-4.062500e-01 -3.750000e-01  8.431375e-02  1.640000e+02  1.770000e+02]


## Create graph 

```

model = new Pointnet(
    mode, string classification | segmentation
    nb_class, int 
    input_channel, int
    nb_pt, int default 1024
    base_laerning_rate, float default 0.001
    gpu_idx, int default 0,
    momentum, float default 0.9
    decay_step, int 200000
    decay_rate, float 0.7
    log_dir, string default logs
    bn_init_decay, float default 0.5
    bn_decay_decay_rate, float 0.5
    bn_decay_decay_step, float 200000
    bn_decay_clip, float 0.9
)

model.train(
files, string[]
batch_size, int default 32
)

model.test(
files, string[]
batch_size, int default 32
)


```

In [12]:
from Pointnet import Pointnet
import tensorflow as tf
import cls_provider as provider
import os
import datetime

tf.reset_default_graph()
ptnet = Pointnet('classification', 40, 3)


INFO:tensorflow:Summary name classify loss is illegal; using classify_loss instead.
INFO:tensorflow:Summary name mat loss is illegal; using mat_loss instead.


In [13]:



config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = False
sess = tf.Session(config=config)
init = tf.global_variables_initializer()

sess.run(init, {ptnet.end_points['is_training_pl']: True})
ptnet.create_log(sess)

BASE_DIR = '/notebooks/'

# ModelNet40 official train/test split
# modelnet40_ply_hdf5_2048
# 
TRAIN_FILES = provider.getDataFiles( \
    os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/train_files.txt'))
TEST_FILES = provider.getDataFiles(\
    os.path.join(BASE_DIR, 'data/modelnet40_ply_hdf5_2048/test_files.txt'))

MAX_EPOCH = 25
for epoch in range(MAX_EPOCH):
    print('**** EPOCH %03d ****' % (epoch))
    
    ptnet.train(sess, TRAIN_FILES)
    ptnet.test(sess, TEST_FILES)

    # Save the variables to disk.
    if epoch % 10 == 0:
        save_path = ptnet.saver.save(sess, os.path.join('log', "model.ckpt"))
        print("Model saved in file: %s" % save_path)

**** EPOCH 000 ****
mean loss: 655.303208
accuracy: 0.657715
----0-----
----1-----
eval mean loss: 6617.013383
eval accuracy: 0.371753
eval avg class acc: 0.260097
Model saved in file: log/model.ckpt
**** EPOCH 001 ****
mean loss: 1836.734909
accuracy: 0.720703
----0-----
----1-----
eval mean loss: 140782.994630
eval accuracy: 0.359984
eval avg class acc: 0.261337
**** EPOCH 002 ****
mean loss: 4025.577370
accuracy: 0.741699
----0-----
----1-----
eval mean loss: 82267.954428
eval accuracy: 0.585227
eval avg class acc: 0.447585
**** EPOCH 003 ****
mean loss: 8568.304924
accuracy: 0.755371
----0-----
----1-----
eval mean loss: 174554.706397
eval accuracy: 0.630276
eval avg class acc: 0.486901
**** EPOCH 004 ****
mean loss: 15265.199905
accuracy: 0.776367
----0-----
----1-----
eval mean loss: 330604.854036
eval accuracy: 0.663149
eval avg class acc: 0.529298
**** EPOCH 005 ****
mean loss: 25933.711972
accuracy: 0.810662
----0-----
----1-----
eval mean loss: 649106.910600
eval accuracy: 0.

KeyboardInterrupt: 